In [1]:
import bs4
import psycopg2
import logging
import lxml
import sys
import requests
import json
import csv

/Users/tanpham/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load data from data.json
data_file = open("data.json")
data_dict = json.load(data_file)

In [3]:
# Create get_logger() function

def get_logger():
    logger = logging.getLogger('my_logger')
    logger.setLevel(logging.DEBUG)
    logger.propagate=False
    
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(lineno)d : %(message)s')
    
    file_handler = logging.FileHandler(data_dict["file_log"])
    file_handler.setFormatter(formatter)
    file_handler.setLevel(logging.DEBUG)

    if not logger.hasHandlers():
        logger.addHandler(file_handler)
        
    return logger
    

In [4]:
# Init csv writer
csv_file = open(data_dict["file_csv"], 'w')
csv_writer = csv.writer(csv_file)

# Reset file log
with open(data_dict["file_log"], 'w'): pass

# Init file log to track history
logger = get_logger()


In [5]:
# Create connection to DBSM

def create_connection():
    f = open('password.txt', 'r')
    pw = f.read()
    f.close()
    connection = psycopg2.connect(
        host = data_dict["dbms_host"],
        database = "postgres",
        user = "postgres",
        password = pw,
        port = "5432"
    )
    connection.set_session(autocommit=True)
    return connection

connection = create_connection()
logger.info(f"Connected to {connection}")

In [6]:
# Create a table in DBMS

fields = data_dict["event"]["data_field"]
table_name = data_dict["table_name"]

def create_table_dbms(connection, table_name, fields):
    logger.info(f"Start create table {table_name}")
    list_field_name_field_type = [f'{field["field"]} {field["type"]} ' for field in fields]
    str_field_name_field_type = ', '.join(list_field_name_field_type)
    create_table_str = f"""
        create table if not exists {table_name} (
            {str_field_name_field_type}
        )
    """
    cur = connection.cursor()
    cur.execute(create_table_str)

def create_table_csv(csv_writer, fields):
    list_field_name = [f'{field["field"]}' for field in fields]
    csv_writer.writerow(list_field_name)

create_table_csv(csv_writer, fields)
create_table_dbms(connection, table_name, fields)